In [8]:
import os
import sys
 
path = os.path.abspath(os.path.join('..'))
sys.path.append(path)

In [2]:
import pandas as pd
from collect_snippets.src.analyze_issues import merge_data_files_into_frame, filter_PRs, filter_commits,\
    PATH_TO_DIR_WITH_PICKLE_DATA

# Загрузка данных об инцидентах всех репозиториев большого списка



In [3]:
whole_data = merge_data_files_into_frame(PATH_TO_DIR_WITH_PICKLE_DATA,
                                         file_type='pickle')
whole_data = whole_data['issues']

# Фильтрация пулл-реквестов

Существует стандартная практика при создании пулл-реквестов, в рамках которой в описании пулл-реквеста дается ссылка на инцидент (issue),
при этом в истории этого инцидента дается ссылка на данный пулл-реквест. Иногда бывает так, что в истории инцидента пулл-реквест упомянут (т.е. есть ссылка на него в timeline этого инцидента),
но в тексте сообщения этого пулл-реквеста ссылка на инцидент отсутствует. Такое может быть по следующим причинам:

- если разработчик пулл-реквеста
не знает о каком-то инциденте, он в тексте сообщения не дает ссылки на инцидент, тогда как в истории инцидента ссылка на этот пулл-реквест присутствует, поскольку связь пулл-реквеста и инцидента была кем-то замечена,
например, самим разработчиком этого пулл-реквеста;

- если пишется пулл-реквест и только потом регистрируется инцидент;

- изменения, входящие в пулл-реквест, могут разрешать также и другие инциденты, а исправление данного инцидента является побочным продуктом данного пулл-реквеста; 

- пулл-реквест ссылается на инцидент на своей странице, но не в своем сообщении непосредственно.

В дальнейшем предполагается использовать только следующие пулл-реквесты со статусом `merged`, упомянутые на странице инцидента:

- закрывающие пулл-реквесты; 

- среди незакрывающих пулл-реквестов те из них, которые ссылаются на обрабатываемые ими инциденты либо в своем сообщении, либо в заголовке.

Код ниже реализует эту фильтрацию. По результатам работы этого кода в данные об отфильтрованных пулл-реквестах добавляется информация обо всех инцидентах, на которые в тексте соообщения пулл-реквеста имеется ссылка. Кроме того, в данные о пулл-реквестах добавляется также инфрмация о токенах (словах), предшествующих появлению этих ссылок в тексте пулл-реквестов. 

Создается новая колонка `keyword closing PRs` содержащая незакрывающие пулл-реквесты, привязанные к инциденту при помощи специальных предопределенных слов [см ссылку](https://docs.github.com/en/github/managing-your-work-on-github/linking-a-pull-request-to-an-issue#linking-a-pull-request-to-an-issue-using-a-keyword) Также создается еще две колонки: `non-closing PRs` и `relevent non-closing PRs`. Первая из этих колонок содержит пулл-реквесты, содержащие ссылку на инцидент (в тексте или заголовке), а вторая - пулл-реквесты, в тексте которых данной ссылке предшествуют некоторые предопределенные слова, дополнительно подтверждающие, что пулл-реквест именно исправляет данную ошибку, а не просто ссылается на данный инцидент.

In [6]:
def at_least_one_PR_treats_several_issues(PRs_infos):
    for PR_info in PRs_infos:
        if len(PR_info['linked_issues']) > 1:
            return True
    return False

Посмотрим на распределение токенов в тексте сообщения незакрывающих пулл-реквестов (колонка `non-closing PRs`), отстоящих от ссылок на инциденты не более, чем на 10 токенов.

In [12]:
def aggregate_PRs_tokens_given_issue(PRs_infos, 
                                     issue_url):
    PRs_messages_tokens = ''
    for PR_info in PRs_infos:
        for tokens in (PR_info['linked_issues'][issue_url] 
                       if issue_url in PR_info['linked_issues'] else []):
            PRs_messages_tokens += ' '.join(tokens) + ' '
    return PRs_messages_tokens

Удалим знаки препинания и стоп-слова. Покажем частоты каждого токена.

In [13]:
from collections import Counter
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/kks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Следующий код собственно реализует поиск наиболее часто встречающихся предшествующих токенов

In [14]:
non_closing_PRs_tokens = whole_data[['non-closing PRs', 'url']].apply(lambda x:
                                                                      aggregate_PRs_tokens_given_issue(x['non-closing PRs'],
                                                                                                       x['url']),
                                                                      axis=1).agg(func='sum')

non_closing_PRs_tokens = non_closing_PRs_tokens.translate(str.maketrans('', '', string.punctuation))

PRs_tokens_counts = Counter(w for w in (non_closing_PRs_tokens).split() 
                            if not w in set(stopwords.words('english')))
PRs_tokens_counts.most_common(100)

[('issue', 5360),
 ('fix', 4088),
 ('fixes', 3006),
 ('pr', 2418),
 ('issues', 2300),
 ('see', 1840),
 ('related', 1574),
 ('reference', 1410),
 ('addresses', 1258),
 ('refs', 1048),
 ('summary', 968),
 ('bugfix', 924),
 ('bug', 882),
 ('feature', 538),
 ('test', 502),
 ('address', 483),
 ('closes', 452),
 ('changes', 436),
 ('fixed', 423),
 ('purpose', 423),
 ('also', 412),
 ('description', 407),
 ('reported', 403),
 ('resolves', 390),
 ('request', 369),
 ('error', 343),
 ('tests', 324),
 ('pull', 316),
 ('add', 310),
 ('described', 308),
 ('available', 307),
 ('part', 301),
 ('fixing', 296),
 ('version', 292),
 ('change', 286),
 ('problem', 272),
 ('number', 272),
 ('solve', 258),
 ('work', 258),
 ('solves', 237),
 ('use', 228),
 ('information', 227),
 ('ref', 226),
 ('new', 224),
 ('like', 220),
 ('workaround', 219),
 ('python', 216),
 ('references', 203),
 ('using', 201),
 ('one', 200),
 ('additional', 197),
 ('module', 193),
 ('mentioned', 192),
 ('file', 192),
 ('added', 183),
 (

# Список пулл-реквестов, содержащих в тексте сообщения заданный токен

In [15]:
def select_PRs_containing_token_given_issue(PRs_infos, 
                                            issue_url, 
                                            token):
    for PR_info in PRs_infos:
        for preceding_tokens in (PR_info['linked_issues'][issue_url] 
                                 if issue_url in PR_info['linked_issues'] else []):
            for preceding_token in preceding_tokens:
                if token in preceding_token:
                    return True
    return False

In [16]:
token = 'backport'
token_whole_data = whole_data.loc[whole_data[['relevant non-closing PRs', 
                                              'url']].apply(lambda x: select_PRs_containing_token_given_issue(x['relevant non-closing PRs'], 
                                                                                                              x['url'],
                                                                                                              token),
                                                            axis=1)]

# Фильтрация коммитов

По тем же принципам, что и для пулл-реквестов, построена также и фильтрация коммитов. Коммиты, особенно те из них, которые находятся внутри пулл-реквестов, могут вообще не содержать никаких ссылок на инцидент с своих заголовках и сообщениях. В коде ниже фильтруются только те коммиты, которые имеют явную ссылку на себя на странице инцидента.

# Фильтрация инцидентов

Посмотрим сколько инцидентов имеют в тексте своего сообщения полный лог ошибки (traceback).

In [9]:
def has_error_message(error_info):
    return any(error_item['piece_type'] == 'error message' for error_item in error_info)


traceback_issues = whole_data.loc[whole_data['source code and errors'].apply(has_error_message)]
traceback_issues.to_pickle(path + '/data/traceback_issues.pickle')

# Оставшиеся вопросы

- надо ли фильтровать по статусу коммита (с `None`, `Success` и `Failure` точно можно, непонятно что с `Pending` или `Error`)

- надо ли отдельно обрабатывать коммит, если содержащий его пулл-реквест тоже планируется обрабатывать для данного инцидента (дать задание Илье делать такую проверку)

- оттестировать код фильтрации коммитов и пулл-реквестов